In [1]:
%cd ..
%pwd

/workspace/DeepSpeech-2.x


'/workspace/DeepSpeech-2.x'

In [2]:
import math
import random
import tarfile
import logging
import numpy as np
from collections import namedtuple
from functools import partial

import paddle
from paddle.io import Dataset
from paddle.io import DataLoader
from paddle.io import BatchSampler
from paddle.io import DistributedBatchSampler
from paddle import distributed as dist



/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


In [3]:
import sys
import argparse
import functools
from deepspeech.utils.utility import add_arguments, print_arguments
parser = argparse.ArgumentParser(description=__doc__)
add_arg = functools.partial(add_arguments, argparser=parser)
# yapf: disable
add_arg('num_samples',      int,    5,     "# of samples to infer.")
add_arg('beam_size',        int,    500,    "Beam search width.")
add_arg('num_proc_bsearch', int,    8,      "# of CPUs for beam search.")
add_arg('num_conv_layers',  int,    2,      "# of convolution layers.")
add_arg('num_rnn_layers',   int,    3,      "# of recurrent layers.")
add_arg('rnn_layer_size',   int,    2048,   "# of recurrent cells per layer.")
add_arg('alpha',            float,  2.5,    "Coef of LM for beam search.")
add_arg('beta',             float,  0.3,    "Coef of WC for beam search.")
add_arg('cutoff_prob',      float,  1.0,    "Cutoff probability for pruning.")
add_arg('cutoff_top_n',     int,    40,     "Cutoff number for pruning.")
add_arg('use_gru',          bool,   False,  "Use GRUs instead of simple RNNs.")
add_arg('use_gpu',          bool,   True,   "Use GPU or not.")
add_arg('share_rnn_weights',bool,   True,   "Share input-hidden weights across "
                                            "bi-directional RNNs. Not for GRU.")
add_arg('unit_type',    str,
        'char',
        "Options: char, word, spm.",
        choices=['char', 'word', 'spm'])
add_arg('spm_model_prefix',    str,
        'examples/tiny/s1/data/spm_bpe',
        "spm model prefix.",)
add_arg('infer_manifest',   str,
        'examples/tiny/s1/data/manifest.tiny',
        "Filepath of manifest to infer.")
add_arg('mean_std_path',    str,
        'examples/tiny/s1/data/mean_std.npz',
        "Filepath of normalizer's mean & std.")
add_arg('vocab_path',       str,
        'examples/tiny/s1/data/vocab.txt',
        "Filepath of vocabulary.")
add_arg('lang_model_path',  str,
        'models/lm/common_crawl_00.prune01111.trie.klm',
        "Filepath for language model.")
add_arg('model_path',       str,
        'examples/tiny/s1/checkpoints/step_final',
        "If None, the training starts from scratch, "
        "otherwise, it resumes from the pre-trained model.")
add_arg('decoding_method',  str,
        'ctc_beam_search',
        "Decoding method. Options: ctc_beam_search, ctc_greedy",
        choices = ['ctc_beam_search', 'ctc_greedy'])
add_arg('error_rate_type',  str,
        'wer',
        "Error rate type for evaluation.",
        choices=['wer', 'cer'])
add_arg('specgram_type',    str,
        'fbank',
        "Audio feature type. Options: linear, mfcc.",
        choices=['linear', 'mfcc'])
add_arg('feat_dim',  int,  80, "mfcc or fbank feat dim.")
add_arg('delta_delta',  bool,  False, "delta delta")
# yapf: disable
args = parser.parse_args([])
print(vars(args))

/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'num_samples': 5, 'beam_size': 500, 'num_proc_bsearch': 8, 'num_conv_layers': 2, 'num_rnn_layers': 3, 'rnn_layer_size': 2048, 'alpha': 2.5, 'beta': 0.3, 'cutoff_prob': 1.0, 'cutoff_top_n': 40, 'use_gru': False, 'use_gpu': True, 'share_rnn_weights': True, 'unit_type': 'char', 'spm_model_prefix': 'examples/tiny/s1/data/spm_bpe', 'infer_manifest': 'examples/tiny/s1/data/manifest.tiny', 'mean_std_path': 'examples/tiny/s1/data/mean_std.npz', 'vocab_path': 'examples/tiny/s1/data/vocab.txt', 'lang_model_path': 'models/lm/common_crawl_00.prune01111.trie.klm', 'model_path': 'examples/tiny/s1/checkpoints/step_final', 'decoding_method': 'ctc_beam_search', 'error_rate_type': 'wer', 'specgram_type': 'fbank', 'feat_dim': 80, 'delta_delta': False}


In [4]:
from deepspeech.frontend.utility import read_manifest
from deepspeech.frontend.augmentor.augmentation import AugmentationPipeline
from deepspeech.frontend.featurizer.speech_featurizer import SpeechFeaturizer
from deepspeech.frontend.speech import SpeechSegment
from deepspeech.frontend.normalizer import FeatureNormalizer


from deepspeech.io.collator import SpeechCollator
from deepspeech.io.dataset import ManifestDataset
from deepspeech.io.sampler import (
    SortagradDistributedBatchSampler,
    SortagradBatchSampler,
)
from deepspeech.io import create_dataloader
batch_reader = create_dataloader(
                      manifest_path=args.infer_manifest,
                      unit_type=args.unit_type,
                      vocab_filepath=args.vocab_path,
                      mean_std_filepath=args.mean_std_path,
                      spm_model_prefix=args.spm_model_prefix,
                      augmentation_config='{}',
                      max_input_len=27.0,
                      min_input_len=0.0,
                      max_output_len=float('inf'),
                      min_output_len=0.0,
                      max_output_input_ratio=float('inf'),
                      min_output_input_ratio=0.0,
                      stride_ms=10.0,
                      window_ms=20.0,
                      max_freq=None,
                      specgram_type=args.specgram_type,
                      feat_dim=args.feat_dim,
                      delta_delta=args.delta_delta,
                      use_dB_normalization=True,
                      random_seed=0,
                      keep_transcription_text=True,
                      is_training=False,
                      batch_size=args.num_samples,
                      num_workers=0,
                      sortagrad=True,
                      shuffle_method=None,
                      dist=False)

/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/scipy/fftpack/__init__.py:103: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from numpy import (exp, inf, pi, sqrt, floor, sin, cos, around, int,
/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/numba/core/

In [5]:
for idx, (audio, audio_len, text, text_len) in enumerate(batch_reader()):
    print('test:', text)
    print("test raw:", ''.join( chr(i) for i in text[0][:int(text_len[0])] ))
    print("test raw:", ''.join( chr(i) for i in text[-1][:int(text_len[-1])] ))
    print('audio len:', audio_len)
    print('test len:', text_len)
    print('audio:', audio)
    break

/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:354: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if arr.dtype == np.object:


test: Tensor(shape=[5, 23], dtype=int32, place=CUDAPinnedPlace, stop_gradient=True,
       [[116, 104, 101,  32, 116, 119, 101, 110, 116, 105, 101, 115, -1 , -1 , -1 , -1 , -1 , -1 , -1 , -1 , -1 , -1 , -1 ],
        [119, 104, 101, 114, 101,  32, 105, 115,  32, 116, 104,  97, 116, -1 , -1 , -1 , -1 , -1 , -1 , -1 , -1 , -1 , -1 ],
        [116, 101, 110,  32, 115, 101,  99, 111, 110, 100, 115, -1 , -1 , -1 , -1 , -1 , -1 , -1 , -1 , -1 , -1 , -1 , -1 ],
        [104, 101,  32, 100, 111, 101, 115, 110,  39, 116,  32, 119, 111, 114, 107,  32,  97, 116,  32,  97, 108, 108, -1 ],
        [119, 104, 101, 114, 101,  32, 105, 115,  32, 109, 121,  32,  98, 114, 111, 116, 104, 101, 114,  32, 110, 111, 119]])
test raw: the twenties
test raw: where is my brother now
audio len: Tensor(shape=[5], dtype=int64, place=CUDAPinnedPlace, stop_gradient=True,
       [163, 173, 184, 190, 203])
test len: Tensor(shape=[5], dtype=int64, place=CUDAPlace(0), stop_gradient=True,
       [12, 13, 11, 22, 23])
audi

In [6]:
keep_transcription_text=False
batch_reader = create_dataloader(
                      manifest_path=args.infer_manifest,
                      unit_type=args.unit_type,
                      vocab_filepath=args.vocab_path,
                      mean_std_filepath=args.mean_std_path,
                      spm_model_prefix=args.spm_model_prefix,
                      augmentation_config='{}',
                      max_input_len=27.0,
                      min_input_len=0.0,
                      max_output_len=float('inf'),
                      min_output_len=0.0,
                      max_output_input_ratio=float('inf'),
                      min_output_input_ratio=0.0,
                      stride_ms=10.0,
                      window_ms=20.0,
                      max_freq=None,
                      specgram_type=args.specgram_type,
                      feat_dim=args.feat_dim,
                      delta_delta=args.delta_delta,
                      use_dB_normalization=True,
                      random_seed=0,
                      keep_transcription_text=keep_transcription_text,
                      is_training=False,
                      batch_size=args.num_samples,
                      num_workers=0,
                      sortagrad=True,
                      shuffle_method=None,
                      dist=False)
for idx, (audio, audio_len, text, text_len) in enumerate(batch_reader()):
    print('test:', text)
    print("test raw:", ''.join( chr(i) for i in text[0][:int(text_len[0])] ))
    print("test raw:", ''.join( chr(i) for i in text[-1][:int(text_len[-1])] ))
    print('test len:', text_len)
    print('audio:', audio)
    print('audio len:', audio_len)
    break

test: Tensor(shape=[5, 23], dtype=int32, place=CUDAPinnedPlace, stop_gradient=True,
       [[87, 37, 26,  1, 87, 97, 26, 61, 87, 38, 26, 82, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [97, 37, 26, 79, 26,  1, 38, 82,  1, 87, 37,  3, 87, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [87, 26, 61,  1, 82, 26, 18, 64, 61, 25, 82, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [37, 26,  1, 25, 64, 26, 82, 61,  2, 87,  1, 97, 64, 79, 52,  1,  3, 87,  1,  3, 53, 53, -1],
        [97, 37, 26, 79, 26,  1, 38, 82,  1, 58, 102,  1, 17, 79, 64, 87, 37, 26, 79,  1, 61, 64, 97]])
test raw: W%Wa=W&R
test raw: a%O&R:fO@W%O=@a
test len: Tensor(shape=[5], dtype=int64, place=CUDAPlace(0), stop_gradient=True,
       [12, 13, 11, 22, 23])
audio: Tensor(shape=[5, 203, 80], dtype=float32, place=CUDAPinnedPlace, stop_gradient=True,
       [[[-51.32406616, -17.91388321,  0.00000000 , ..., -26.66350746, -27.46039391, -27.22303963],
         [-15.19027233, -20.52460480,  0.0000